In [1]:
pip install dask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 20.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 12.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import io
import os
from google.cloud import vision
from pdf2image import convert_from_path
import dask

# Set up Vision client 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'savvy-hybrid-389017-984cabbf64b0.json'
poppler = "/project/arcc-students/csloan5/environments/GPU_env/bin/"
client = vision.ImageAnnotatorClient()

# Top level folder containing subfolders of PDFs 
root_folder = '/project/arcc-students/OilWellProject2023.Summer/vert_pages'
output_folder = 'jsons/Testing/'

def ocr_pdf(pdf_path, output_path):
    """OCR a single PDF"""
    images = convert_from_path(pdf_path,poppler_path=poppler)
    image = images[0]
    buf = io.BytesIO()
    image.save(buf, format='JPEG')
    byte_im = buf.getvalue()
    image = vision.Image(content=byte_im)
    response = client.document_text_detection(image=image)

    json_string = type(response).to_json(response)
    text_path = os.path.splitext(pdf_path)[0] + '.json'
    output_path = os.path.join(output_folder, text_path)
    print(output_path)
    with open(output_path, 'w') as text_file:
        print(json_string, file=text_file)

    
def process_folder(folder_path):
    """Process a single subfolder"""
    output_folder =  os.path.join(output_root, folder_path.split(os.sep)[-1])
    os.makedirs(output_folder, exist_ok=True)
  
    for root, files in os.walk(folder_path):
        for pdf_path in files:
            if pdf_path.endswith('.pdf'):
                pdf_full_path = os.path.join(root, pdf_path)
                filename = os.path.splitext(pdf_path)[0]
                output_path = os.path.join(output_folder, f"{filename}.json")
                dask.delayed(ocr_pdf)(pdf_full_path, output_path)
  

In [8]:
      
if __name__ == '__main__':

    folders = [os.path.join(root_folder, f) for f in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, f))]

    output_root = 'jsons/Testing/'

    tasks = [dask.delayed(process_folder)(f) for f in folders]

    dask.compute(*tasks)

    print('OCR complete for all PDFs')

hellohello

hello
OCR complete for all PDFs


In [20]:
ocr_pdf('TestN/Temp2/bruh.pdf','jsons/Testing')

bruh
jsons/Testing


IsADirectoryError: [Errno 21] Is a directory: 'jsons/Testing'